In [ ]:
import kfp
from kfp import dsl
from kfp.dsl import pipeline
from kfp.dsl import (Artifact,
                     Dataset,
                     Input,
                     Model,
                     Output,
                     Metrics,
                     ClassificationMetrics,
                     component,
                     OutputPath,
                     InputPath)

from kfp import compiler
import google.cloud.aiplatform as aip
#from google_cloud_pipeline_components.v1.vertex_notification_email import VertexNotificationEmailOp
from datetime import datetime

#https://www.kubeflow.org/docs/components/pipelines/user-guides/core-functions/control-flow/

In [ ]:
title = 'test-demo'

now_string = datetime.now().strftime("%Y%m%d%H%M%S")
job_name_id = f'{title}-{now_string}'

PROJECT_ID = 'test-project'
REGION = 'us-central1'
BUCKET_NAME = 'test-project-main'
BUCKET_URI = f'gs://{BUCKET_NAME}'

PIPELINE_ROOT = f"{BUCKET_URI}"
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

In [ ]:
#@component(base_image="", install_kfp_package=False)
@component()

def data_a(
):
    import pandas as pd
    import numpy as np
    import os
    from google.cloud import bigquery
    from google.cloud import aiplatform as aip
    from google.cloud import storage
    
    project_number = os.environ["CLOUD_ML_PROJECT_ID"]
    bqclient = bigquery.Client(project=project_number)
                                
    table = biguqery.TableReference.from_string("bigquery-public-data.new_york.311_service_requests")
    row = bqclient.list_rows(table=table)
    df = rows.to_dataframe(create_bqstorage_client=True)
    print(df.shape)

In [ ]:
#@component(base_image="", install_kfp_package=False)
@component()

def data_b(
):
    import pandas as pd
    import numpy as np
    import os
    from google.cloud import bigquery
    from google.cloud import aiplatform as aip
    from google.cloud import storage
    
    project_number = os.environ["CLOUD_ML_PROJECT_ID"]
    bqclient = bigquery.Client(project=project_number)
                                
    sql = """
    SELECT * FROM `bigquery-public-data.new_york.311_service_requests`
    """
    df = bqclient.query(sql).to_dataframe()
    print(df.shape)

In [ ]:
@pipeline(name = "demo-test",
          description = "putting together some code for pipelines",
          pipeline_root = PIPELINE_ROOT
         )
def pipeline_demo():
    
    data_a_task = data_a()
    data_a_task.set_memory_limit('10G')
    data_a_task.set_cpu_limit('10')
    data_a_task.set_caching_options(False)
    
    data_b_task = data_b().after(data_a_task)
    data_b_task.set_memory_limit('10G')
    data_b_task.set_cpu_limit('10')
    data_b_task.set_caching_options(False)

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline_demo,
    package_path="test.yaml"
)

In [ ]:
now = datetime.now()
dt_string = now.strftime("%Y%m%d%H%M%S")
job_name_id = f'demo-test-{dt_string}'

In [ ]:
print(job_name_id)

In [ ]:
job = aip.PipelineJob(
    display_name = "test-demo",
    template_path = "test.yaml",
    job_id = job_name_id,
    pipeline_root = PIPELINE_ROOT
)

job.run()